In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import math
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)
import datetime as dt
from conversight import Dataset, Flow, Parameter, SmartAnalytics,task, ProactiveInsights,TaskLibrary, FlowLibrary, Context
from tqdm import tqdm

In [3]:
tsk = TaskLibrary()

Tasks loaded  !!


In [4]:
tsk.t1.get_df

#,Version,Description,Type,Sub Type,Level,Inputs,Outputs,Registered ORG,Slug
1,0.2,Task description goes here.,generic,Function,O,"query: str, ds_id: str",Attribute,Aev-Conversions,5014fbdc-e065-4ca4-8763-7a9f86be9d3e


In [5]:
from csContext import CSContext

In [6]:
@task
def frcst_preprocessing(ctx: CSContext, df: pd.core.frame.DataFrame, key_col: str, dat_col: str, mat_col: str, F_Mode: str,
                  outlier_flag_by_product: bool, imputation_method: str, outlier_threshold: str, global_smoothing: bool)->dict:
    """
    Write the doc string here, Refer Forecasting Model task
    """
    try:
        import warnings
        warnings.filterwarnings("ignore")
        import pandas as pd
        import numpy as np
        import datetime as dt
        from conversight import Dataset,Context
        # from conversight import TaskPool
        # tp = TaskPool(pool_size=480)
        # ctx=Context()
        ctx.log.info("preprocessing is started ->")
        ctx.log.info(f"selected forecast mode : {F_Mode}")
        outlier_threshold = float(outlier_threshold)
    
        def impute_outliers_by_product(df, key_column, value_column, imputation_value, threshold):
            ctx.log.info(f"Outliers deduction by Product method {imputation_value} with threshold {threshold}")
            def impute_outliers_group(group):
                group[value_column] = group[value_column].apply(lambda x: max(0, x))
                Q1 = group[value_column].quantile(0.25)
                Q3 = group[value_column].quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - threshold * IQR
                upper_bound = Q3 + threshold * IQR
                outliers_mask = (group[value_column] < lower_bound) | (group[value_column] > upper_bound)
                if imputation_value.lower() == "mean":
                    imputed_value = group[value_column].mean()
                elif imputation_value.lower() == "median":
                    imputed_value = group[value_column].median()
                elif imputation_value.lower() == "mode":
                    imputed_value = group[value_column].mode().iloc[0]
                elif "qv" in imputation_value.lower():
                    imputation_value_qv = imputation_value.split("=")[1]
                    qv = float(imputation_value_qv)
                    qv = group[value_column].quantile(qv)
                    imputed_value = qv
                else:
                    iv = float(imputation_value)
                    imputed_value = iv
                group[value_column] = group[value_column].mask(outliers_mask, imputed_value)
                return group
            ctx.log.info("Identifying and imputing outliers..")
            df = df.groupby(key_column, observed=True).apply(impute_outliers_group)
            return df

        def impute_outliers(df, column_name, imputation_value, threshold):
            ctx.log.info(f"Outliers deduction method {imputation_value} with threshold {threshold}")
            df[column_name] = df[column_name].apply(lambda x: max(0, x))
            Q1 = df[column_name].quantile(0.25)
            Q3 = df[column_name].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - threshold * IQR
            upper_bound = Q3 + threshold * IQR
            outliers = df[(df[column_name] < lower_bound) | (df[column_name] > upper_bound)]
            if imputation_value.lower() == "mean":
                imputed_value = df[column_name].mean()
            elif imputation_value.lower() == "median":
                imputed_value = df[column_name].median()
            elif imputation_value.lower() == "mode":
                imputed_value = df[column_name].mode().iloc[0]
            elif "qv" in imputation_value.lower():
                imputation_value_qv = imputation_value.split("=")[1]
                qv = float(imputation_value_qv)
                qv = df[column_name].quantile(qv)
                imputed_value = qv
            else:
                iv = float(imputation_value)
                imputed_value = iv
            ctx.log.info("Identifying and imputing outliers..")
            df[column_name] = df[column_name].mask((df[column_name] < lower_bound) | (df[column_name] > upper_bound), imputed_value)
            return df

        def missed_data(df,keys,dat_col,mat_col,i,min_date,max_date,F_Mode):
            df1 = df[df[keys]==i].reset_index(drop=True)
            df1[dat_col] = pd.to_datetime(df1[dat_col]).dt.floor("D")
            df1 = df1.sort_values([dat_col])
            training = df1.groupby(dat_col, observed=True)[mat_col].sum().reset_index()
            if F_Mode.lower() == "week":
                dates_required = pd.date_range(min_date, max_date, freq="W-MON", inclusive="left").strftime("%Y-%m-%d").tolist()
            elif F_Mode.lower() == "month":
                dates_required = pd.date_range(min_date, max_date, freq="MS").strftime("%Y-%m-01").tolist()
            dates_required = pd.DataFrame(dates_required, columns = [dat_col])
            dates_required[dat_col] = pd.to_datetime(dates_required[dat_col]).dt.floor("D")
            processed_data = dates_required.merge(training, on=dat_col, how="left")
            processed_data.columns = [dat_col, mat_col]
            processed_data = processed_data.fillna(0)
            processed_data[keys] = i
            processed_data[dat_col] = pd.to_datetime(processed_data[dat_col])
            return processed_data

        def missed_data_faster(df, keys, dat_col, mat_col, min_date, max_date, F_Mode):
            df[dat_col] = pd.to_datetime(df[dat_col]).dt.floor("D")
            if F_Mode.lower() == "month":
                #all_dates = pd.date_range(min_date, max_date, freq="MS", inclusive="left")
                all_dates = pd.date_range(min_date, max_date, freq="MS", inclusive="both")
            elif F_Mode.lower() == "week":
                all_dates = pd.date_range(min_date, max_date, freq="W-MON", inclusive="both")
            #all_dates_df = pd.DataFrame({dat_col: all_dates})
            df.set_index([keys, dat_col], inplace=True)
            df_unstacked = df.unstack(level=0).reindex(all_dates).stack(dropna=False).reset_index()
            df_unstacked[mat_col].fillna(0, inplace=True)
            df_unstacked.rename(columns={'level_0': dat_col}, inplace=True)
            return df_unstacked
            
        def missed_data_optimized(df, keys, dat_col, mat_col, min_date, max_date, F_Mode):
            df[dat_col] = pd.to_datetime(df[dat_col]).dt.floor("D")
            if F_Mode.lower() == "month":
                dates_required = pd.date_range(min_date, max_date, freq="MS").strftime("%Y-%m-01").tolist()
            elif F_Mode.lower() == "week":
                dates_required = pd.date_range(min_date, max_date, freq="W-MON").strftime("%Y-%m-%d").tolist()
            dates_required = pd.DataFrame(dates_required, columns=[dat_col])
            dates_required[dat_col] = pd.to_datetime(dates_required[dat_col]).dt.floor("D")
            unique_keys = df[keys].unique()
            all_combinations = pd.MultiIndex.from_product([unique_keys, dates_required[dat_col]], names=[keys, dat_col]).to_frame(index=False)
            grouped = df.groupby([keys, dat_col], observed=True)[mat_col].sum().reset_index()
            processed_data = all_combinations.merge(grouped, on=[keys, dat_col], how="left")
            processed_data[mat_col] = processed_data[mat_col].fillna(0)
            return processed_data
        
        #### Merging all key column and drop null values for the main columns
        columns = key_col.split(",")
        keys = "key_column"
        df[keys] = df[columns].astype(str).agg("_".join, axis=1)
        columns.append(keys)
        columns.extend([dat_col, mat_col])
        df = df.dropna(subset=columns).reset_index(drop=True)

        #### Drop duplicate
        df = df.drop_duplicates()
    
        #### select necessary columns and Convert date to month start or monday start week date
        df = df[[dat_col, mat_col, keys]]
        df[keys] = df[keys].astype(str)
        df[dat_col] = pd.to_datetime(df[dat_col]).dt.floor("D")
        if F_Mode.lower() == "month":
            df[dat_col] = ((df[dat_col] + pd.offsets.MonthEnd(0) - pd.offsets.MonthBegin(1)).dt.floor("D"))
        elif F_Mode.lower() == "week":
            df[dat_col] = df[dat_col].dt.to_period("W").apply(lambda r: r.start_time)
        df = df.groupby([dat_col, keys], observed=True)[mat_col].sum().reset_index()

        #### Remove Current month/week data from training
        if F_Mode.lower() == "month":
          P = dt.datetime.today().replace(day=1).strftime("%Y-%m-%d")
          df = df[df[dat_col] < P].reset_index(drop=True)
        elif F_Mode.lower() == "week":
          #current week start monday
          current_date = dt.datetime.today()
          P = (current_date - dt.timedelta(days=current_date.weekday())).strftime("%Y-%m-%d")
          df = df[df[dat_col] < P].reset_index(drop=True)
        
        #### remove nagetive values
        df[mat_col] = df[mat_col].apply(lambda x: max(0, x))

        #### Remove outliers from the training data
        if outlier_flag_by_product and global_smoothing:
            ctx.log.error("Both variables are True, set according...")
            return {"status": "Failed", "message": "processing failed due to bad configuration"}
        else:
            if outlier_flag_by_product:
                ctx.log.info("outlier_flag_by_product is started ->")
                df = impute_outliers_by_product(df, keys, mat_col, imputation_method, outlier_threshold)
                ctx.log.info("outlier deduction is ended ->")
            if global_smoothing:
                ctx.log.info("global_smoothing is started ->")
                df = impute_outliers(df, mat_col, imputation_method, outlier_threshold)
                ctx.log.info("outlier deduction is ended ->")
    
        # Find maximum date and Minimum date
        min_date_limit, max_date_limit = df[dat_col].min(), df[dat_col].max()
        ctx.log.info("Min_date_limit and Max_date_limit calculated successfully")
        ctx.log.info(f"Min - Date --> {min_date_limit} and Max - Date --> {max_date_limit}")
        
        #### Find the missing month/week date and fill with value column by zero
        prod_ids = df[keys].unique()
        ctx.log.info("Getting All unique Materials to do Forecast and Total unique count {}".format(len(prod_ids)))
        # frame, cnt = [], 1
        # for i in  prod_ids:
        #     #df1 = missed_data(df,keys,dat_col,mat_col,i,min_date_limit,max_date_limit,F_Mode)
        #     tp.execute([df,keys,dat_col,mat_col,i,min_date_limit,max_date_limit,F_Mode], missed_data)
        #     if cnt % 100 == 0:
        #         frame.extend(tp.collect_all()[1])
        #     if cnt%500 == 0:
        #         ctx.log.info("Completed {} Items".format(cnt))
        #     cnt+=1
        # # Collect all results after the loop has finished
        # remaining_results = tp.collect_all()[1]
        # frame.extend(remaining_results)
        # result = pd.concat(frame)
        # result = result.reset_index(drop=True)
        result = missed_data_optimized(df, keys, dat_col, mat_col, min_date_limit, max_date_limit, F_Mode)
        result[mat_col]=result[mat_col].apply(lambda x:round(x,2))
    
        ctx.log.info("preprocessing is ended ->")
        parameter = {"dateID":dat_col, "keyID":keys, "matricID":mat_col, "DATA":result, "F_Mode":F_Mode}
        return parameter

    except Exception as e:
        return {"status": "Failed", "message": str(e)}
        

In [7]:
frcst_preprocessing.register(libraryName = "t1")

[2024-07-01 13:15:11,886] [INFO] frcst_preprocessing has been successfully registered. The most recent version available is 0.4 !!


In [8]:
tsk.reload()

Tasks loaded  !!


In [9]:
tsk.t1.frcst_preprocessing.promote("O")

Version 0.4 for the task frcst_preprocessing has been successfully promoted to O  !!
Tasks loaded  !!


In [10]:
tsk.t1.frcst_preprocessing

#,Version,Description,Type,Sub Type,Level,Inputs,Outputs,Registered ORG,Slug
1,0.1,Task description goes here.,generic,Function,O,"df: pd.core.frame.DataFrame, key_col: str, dat_col: str, mat_col: str, F_Mode: str, outlier_flag_by_product: bool, imputation_method: str, outlier_threshold: str, global_smoothing: bool",Name,Aev-Conversions,b5f0aa10-808d-42c4-97a8-8625602689fd
2,0.2,Task description goes here.,generic,Function,O,"df: pd.core.frame.DataFrame, key_col: str, dat_col: str, mat_col: str, F_Mode: str, outlier_flag_by_product: bool, imputation_method: str, outlier_threshold: str, global_smoothing: bool",Name,Aev-Conversions,b5f0aa10-808d-42c4-97a8-8625602689fd
3,0.3,Task description goes here.,generic,Function,O,"df: pd.core.frame.DataFrame, key_col: str, dat_col: str, mat_col: str, F_Mode: str, outlier_flag_by_product: bool, imputation_method: str, outlier_threshold: str, global_smoothing: bool",Name,Aev-Conversions,b5f0aa10-808d-42c4-97a8-8625602689fd
4,0.4,Task description goes here.,generic,Function,O,"ctx: CSContext, df: pd.core.frame.DataFrame, key_col: str, dat_col: str, mat_col: str, F_Mode: str, outlier_flag_by_product: bool, imputation_method: str, outlier_threshold: str, global_smoothing: bool",Name,Aev-Conversions,b5f0aa10-808d-42c4-97a8-8625602689fd


In [12]:
@task
def autoArimaForecastParams(ctx: CSContext, p: dict, dataset_id:str, table_name:str):
    """
    Write the doc string here, Refer Forecasting Model task
    Final result dataframe column names are: key_column,p,d,q,sP,sD,sQ,m and seasonal
    Final table name : AutoArima_Param
    """
    try:
        # import library
        import os
        import sys
        import warnings
        warnings.filterwarnings("ignore")
        import pandas as pd
        import datetime as dt
        from conversight import Dataset,Context
        from conversight import TaskPool
        tp = TaskPool(pool_size=480)
        # ctx=Context()
        ctx.log.info("autoArima Model Forecast checking is started ->")
        
        # do necesaary part for training
        dat_col, mat_col, key_col = p["dateID"], p["matricID"], p["keyID"]
        fieldMapping = {"date_column":dat_col, "value_column":mat_col}
        df = p["DATA"]
        F_Mode = p["F_Mode"]
        
        # training and validation *use suppress logs
        class suppress_logs(object):
            """
            A context manager for doing a "deep suppression" of stdout and stderr in Python, i.e. will suppress all print, even if the print originates
            in a compiled C/Fortran sub-function. This will not suppress raised exceptions, since exceptions are printed to stderr just before a script exits,
            and after the context manager has exited (at least, I think that is why it lets exceptions through).
            """
            def __init__(self):
                self.null_fds = [os.open(os.devnull, os.O_RDWR) for x in range(2)]
                self.save_fds = [os.dup(1), os.dup(2)]
            def __enter__(self):
                os.dup2(self.null_fds[0], 1)
                os.dup2(self.null_fds[1], 2)
            def __exit__(self, *_):
                os.dup2(self.save_fds[0], 1)
                os.dup2(self.save_fds[1], 2)
                for fd in self.null_fds + self.save_fds:
                    os.close(fd)
                    
        def forecastmodel2(df, prod_id, keys, F_Mode):
            try:
                from pandas.tseries.offsets import DateOffset
                from statsmodels.tsa.stattools import adfuller
                import statsmodels.api as sm
                from statsforecast import StatsForecast
                from statsforecast.models import AutoARIMA
                from statsforecast.arima import arima_string
                from scipy.stats import kruskal

                df_model = df[df[keys] == prod_id]
                df_model.drop([keys],axis = 1, inplace = True)
                df_model = df_model.reset_index(drop=True)

                def seasonality_test(series, F_Mode):
                    if F_Mode.lower() == "month":
                        m, fq = 12, "MS"
                    elif F_Mode.lower() == "week":
                        m, fq = 52, "W-MON"
                    idx = np.arange(len(series.index)) % m
                    H_statistic, p_value = kruskal(series, idx)
                    if p_value <= 0.05:
                        seasonal = True
                    else: seasonal = False
                    return m, fq, seasonal
                
                def modeSeasonality(series, F_Mode):
                    from scipy.stats import bartlett, levene
                    if F_Mode.lower() == "month":
                        sfq = 12
                    elif F_Mode.lower() == "week":
                        sfq = 52
                    moving_average = series.rolling(window=sfq, center=True).mean()
                    seasonal_ratios = series / moving_average
                    seasonal_ratios = seasonal_ratios.dropna()
                    group1 = seasonal_ratios[:len(seasonal_ratios)//2]
                    group2 = seasonal_ratios[len(seasonal_ratios)//2:]
                    
                    # Perform Levene's test
                    levene_stat, levene_p_value = levene(group1, group2)
                    if levene_p_value > 0.05:
                        smode = 'multiplicative'
                    else:
                        smode = 'additive'
                    return smode
                

                df_model["unique_id"] = 1
                df_model.columns = ["ds","y","unique_id"]
                sl, fq, seas = seasonality_test(df_model["y"], F_Mode)
                smode = modeSeasonality(df_model['y'], F_Mode)
                sf = StatsForecast(df=df_model,
                                models = [AutoARIMA(season_length = sl, seasonal=seas,
                                                    start_p=0, start_q=0, start_P=0, start_Q=0,
                                                    max_p= 5, max_q=5, max_P=3, max_Q=3,
                                                    max_order=20, max_d=2, #max_D=2,
                                                    # test="adf", seasonal_test="ocsb",
                                                    )
                                            ],freq = fq, n_jobs=-1
                                )
                with suppress_logs():
                    sf.fit()
                #print(arima_string(sf.fitted_[0,0].model_))
                #print(sf.fitted_[0][0].model_["arma"])
                res = sf.fitted_[0][0].model_["arma"]
                p, q, P, Q, m, d, D = res[0], res[1], res[2], res[3], res[4], res[5], res[6]
                if P==0 and Q==0 and D==0:
                    m=0
                data = {keys:[prod_id], "p":[p], "d":[d], "q":[q], "sP":[P], "sD":[D], "sQ":[Q], "m":[m],
                        "seasonal":[seas],"seasonalMode":[smode]}
                result = pd.DataFrame(data)
                del sf, df_model
                return result
            except:
                ctx.log.info("**AutoArima Paramets failed for Item: {}".format(prod_id))

        ##### Main forecast execution start #######################################################
        prod_ids = df[key_col].unique()
        ctx.log.info("Getting All unique Materials to do Forecast and Total unique count {}".format(len(prod_ids)))
        frame = []  
        cnt = 1
        for i in prod_ids:
            #frame.append(forecastmodel2(df, i, key_col, F_Mode))
            tp.execute([df, i, key_col, F_Mode], forecastmodel2)
            if cnt % 100 == 0:
                frame.extend(tp.collect_all()[1])
            if cnt%500 == 0:
                ctx.log.info("Completed {} Items".format(cnt))
            cnt+=1
        # Collect all results after the loop has finished
        remaining_results = tp.collect_all()[1]
        frame.extend(remaining_results)
        
        result_df = pd.concat(frame)
        result_df = result_df.reset_index(drop=True)
                
        # load back to db
        ctx.log.info("autoArima Model Forecast Parameters checking Done")
        ctx.waitUntilDatasetActive(datasetIds=[dataset_id], maxRetries=4, maxWaitTime=15,incrementMultiplier=3)
        ds = Dataset(dataset_id)
        ctx.log.info("Dataset Loaded")
        response = ds.smartAnalytics.create(table_name, result_df, False, True, True)
        if "status" in response and response["status"] == "success":
            ctx.log.info("smart analytics creation is done -> {}".format(dt.datetime.utcnow()))
            ctx.log.info("Creation of table {} was done properly and smart analytics is created successfully".format(table_name))
        else:
            ctx.log.error("smart analytics creation is failed -> {}".format(dt.datetime.utcnow()))
            return {"status": "failed", "message": "Organization processing failed due to an error {}".format(response["message"] if "message" in response else "internalServerError")}

        return {"status": "success", "message": "Forecasting Done Successfully"}

    except Exception as e:
        return {"status": "Failed", "message": str(e)}

In [13]:
autoArimaForecastParams.register(libraryName = "t1")

[2024-07-01 13:22:34,385] [INFO] autoArimaForecastParams has been successfully registered. The most recent version available is 0.4 !!


In [14]:
tsk.reload()

Tasks loaded  !!


In [15]:
tsk.t1.autoArimaForecastParams.promote("O")

Version 0.4 for the task autoArimaForecastParams has been successfully promoted to O  !!
Tasks loaded  !!


In [16]:
tsk.t1.autoArimaForecastParams

#,Version,Description,Type,Sub Type,Level,Inputs,Outputs,Registered ORG,Slug
1,0.1,Task description goes here.,generic,Function,O,"p: dict, dataset_id: str, table_name: str",str,Aev-Conversions,af73efaa-77aa-4a3b-a23e-582ac12afa09
2,0.2,Task description goes here.,generic,Function,O,"p: dict, dataset_id: str, table_name: str",str,Aev-Conversions,af73efaa-77aa-4a3b-a23e-582ac12afa09
3,0.3,Task description goes here.,generic,Function,O,"p: dict, dataset_id: str, table_name: str",str,Aev-Conversions,af73efaa-77aa-4a3b-a23e-582ac12afa09
4,0.4,Task description goes here.,generic,Function,O,"ctx: CSContext, p: dict, dataset_id: str, table_name: str",str,Aev-Conversions,af73efaa-77aa-4a3b-a23e-582ac12afa09


In [17]:
flw = FlowLibrary()

Flows loaded !!


In [18]:
with Flow(name="autoArimaForecastParams", description= "to do Forecast") as flow:
    query = """
    SELECT cast(m_actual_date as date) as "actual_date",
    m_sales_quantity as "sales_quantity", m_product_num as "product_num"
    FROM #FRCST_TEST
    """
    ds_id = "662ba9e1-BZoSk0BIm"
    idp = Parameter("Dataset Id", ds_id)
    q = Parameter("Sql Query", query)
    k = Parameter("""Name of key column (the column that needs to forecasted like product id, It can also
                  be multi-level like warehouse and then product id)""","product_num")
    d = Parameter("Name of Date Column", "actual_date")
    v = Parameter("Name of Metric Column", "sales_quantity")
    tabel_name = Parameter("Name of Forecast Table", "AutoArima_Param")
    F_Mode = Parameter("Define Forecast Mode : week/month", "month")
    im = Parameter("Outlier Imputation Method : mean/median/mode/quartile value(qv=0.95)", "median")
    ot = Parameter("Outlier Threshold", "1.5")
    ofp = Parameter("Outlier Imputation Flag (set yes if Product Level)(if yes then put global flag be false)", True)
    gs = Parameter("""Global Imputation Smoothing (set yes if you have atleast one off product with very high values)
                    (if yes then put outlier flag be false)""", False)
    
    
    a = tsk.t1.get_df(q, idp)
    get_para = tsk.t1.frcst_preprocessing(a, k, d, v, F_Mode, ofp, im, ot, gs)
    fore = tsk.t1.autoArimaForecastParams(get_para, idp, tabel_name)

In [19]:
flow.register(libraryName = "Forecasting_v1",flowName ="autoArimaForecastParams",description = "to do autoArima Paramater check")

'The flow autoArimaForecastParams has been registered successfully. Latest version available now is 0.3 !!'

In [20]:
flw.reload()

Flows loaded !!


In [21]:
flw.Forecasting_v1.autoArimaForecastParams.promote("P")

'Version 0.3 for the flow autoArimaForecastParams has been promoted to P successfully !!'

In [20]:
#flw.Forecasting_v1.autoArimaForecastParams.deleteVersion(0.1)